# 1. Setup

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import sys
import os
import io

import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm

import dask.dataframe as dd

# Datetime handling
import datetime

#Only Applicable for Google Cloud Platform usage.
#import google
#from google.cloud import storage
#from io import StringIO
#from io import BytesIO  

In [3]:
#Only Applicable for Google Cloud Platform usage.
#client = storage.Client()
#bucket = client.get_bucket('sdymain')
#blob = bucket.get_blob(f"BC5dataraw.csv")
#bt = blob.download_as_bytes()
#data = pickle.load(BytesIO(bt))

# 2. Data Load

In [4]:
data = pd.read_csv('NOVAIMS_MAA_2020e21_BusinessCasesDataScience_MindOverData_RetailChallenge.csv', nrows = 10)
data

ProductFamily_ID ProductCategory_ID   ProductBrand_ID   ProductName_ID  \
0        Family_16        Category_11  ProductBrand_306  ProductName_649   
1        Family_16        Category_11  ProductBrand_306  ProductName_649   
2        Family_16        Category_11  ProductBrand_306  ProductName_649   
3        Family_16        Category_11  ProductBrand_306  ProductName_649   
4        Family_16        Category_11  ProductBrand_306  ProductName_649   
5        Family_16        Category_11  ProductBrand_306  ProductName_649   
6        Family_16        Category_11  ProductBrand_306  ProductName_649   
7        Family_16        Category_11  ProductBrand_306  ProductName_649   
8        Family_16        Category_11  ProductBrand_306  ProductName_649   
9        Family_16        Category_11  ProductBrand_306  ProductName_649   

  ProductPackSKU_ID Point-of-Sale_ID        Date         Measures  Value  
0   ProductSKU_1970            POS_1  2017-03-04   Sell-out units      2  
1   ProductSKU_1970            POS_1  2017-03-04  Sell-out values   1540  
2   ProductSKU_1970            POS_1  2016-05-02   Sell-out units      4  
3   ProductSKU_1970            POS_1  2016-05-02  Sell-out values   3080  
4   ProductSKU_1970            POS_1  2016-10-24   Sell-out units      2  
5   ProductSKU_1970            POS_1  2016-10-24  Sell-out values   1540  
6   ProductSKU_1970            POS_1  2017-10-13   Sell-out units      2  
7   ProductSKU_1970            POS_1  2017-10-14   Sell-out units      2  
8   ProductSKU_1970            POS_1  2017-10-15   Sell-out units      2  
9   ProductSKU_1970            POS_1  2017-10-13  Sell-out values   1620

In [5]:
data = pd.read_csv('NOVAIMS_MAA_2020e21_BusinessCasesDataScience_MindOverData_RetailChallenge.csv', nrows = 0)
# The datatypes below actually doesn't impact the data when loaded into the variable (e.g., if there is a float value in Value data, the datatype will automatically be changed to float.)
data = data.astype(
    {
        'ProductFamily_ID':'int',
        'ProductCategory_ID':'int',
        'ProductBrand_ID':'int',
        'ProductName_ID':'int',
        'ProductPackSKU_ID':'int',
        'Point-of-Sale_ID':'int',
        'Date':'datetime64[ns]',
        'Measures':'bool',
        'Value':'float'
    }
)

In [6]:
data

Empty DataFrame
Columns: [ProductFamily_ID, ProductCategory_ID, ProductBrand_ID, ProductName_ID, ProductPackSKU_ID, Point-of-Sale_ID, Date, Measures, Value]
Index: []

In [7]:
for chunk in tqdm(pd.read_csv('NOVAIMS_MAA_2020e21_BusinessCasesDataScience_MindOverData_RetailChallenge.csv', chunksize = 5000000)):
    # for each column of the first six, extract the number and store it as an int
    for i, column in enumerate(chunk.columns[:6]):
        chunk[column] = chunk[column].apply(lambda s: s.split('_')[1]).astype(int)

   # convert Measures to bool
    chunk['Measures'] = chunk['Measures'].replace('Sell-out units', 0)
    chunk['Measures'] = chunk['Measures'].replace('Sell-out values', 1)
    chunk['Measures'] = chunk['Measures'].astype(bool)

    chunk['Date'] = pd.to_datetime(chunk['Date']) 

    data = data.append(chunk, ignore_index = True)
    #We will check the nature of the duplicates before dropping them
    #data = data.drop_duplicates()

37it [12:07, 19.67s/it]


In [8]:
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182342304 entries, 0 to 182342303
Data columns (total 9 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ProductFamily_ID    int64         
 1   ProductCategory_ID  int64         
 2   ProductBrand_ID     int64         
 3   ProductName_ID      int64         
 4   ProductPackSKU_ID   int64         
 5   Point-of-Sale_ID    int64         
 6   Date                datetime64[ns]
 7   Measures            bool          
 8   Value               float64       
dtypes: bool(1), datetime64[ns](1), float64(1), int64(6)
memory usage: 11.0 GB


In [9]:
data.max()

ProductFamily_ID                       21
ProductCategory_ID                    178
ProductBrand_ID                      1535
ProductName_ID                       2850
ProductPackSKU_ID                    8660
Point-of-Sale_ID                      410
Date                  2019-11-01 00:00:00
Measures                             True
Value                              662575
dtype: object

In [10]:
data.min()

ProductFamily_ID                        1
ProductCategory_ID                      1
ProductBrand_ID                         1
ProductName_ID                          1
ProductPackSKU_ID                       1
Point-of-Sale_ID                        1
Date                  2016-01-01 00:00:00
Measures                            False
Value                                 -10
dtype: object

In [11]:
# Product Family, Category, etc, are not actually numeric in nature, but categorical. However, object datatype uses more memory. Hence, the data will be loaded as integers to save memory, but later converted to object type when necessary for certain analysis.
data = data.astype(
    {
        'ProductFamily_ID':np.uint8,
        'ProductCategory_ID':np.uint8,
        'ProductBrand_ID':np.uint16,
        'ProductName_ID':np.uint16,
        'ProductPackSKU_ID':np.uint16,
        'Point-of-Sale_ID':np.uint16
    }
)

In [12]:
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182342304 entries, 0 to 182342303
Data columns (total 9 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ProductFamily_ID    uint8         
 1   ProductCategory_ID  uint8         
 2   ProductBrand_ID     uint16        
 3   ProductName_ID      uint16        
 4   ProductPackSKU_ID   uint16        
 5   Point-of-Sale_ID    uint16        
 6   Date                datetime64[ns]
 7   Measures            bool          
 8   Value               float64       
dtypes: bool(1), datetime64[ns](1), float64(1), uint16(4), uint8(2)
memory usage: 4.6 GB


In [13]:
data.max()

ProductFamily_ID                       21
ProductCategory_ID                    178
ProductBrand_ID                      1535
ProductName_ID                       2850
ProductPackSKU_ID                    8660
Point-of-Sale_ID                      410
Date                  2019-11-01 00:00:00
Measures                             True
Value                              662575
dtype: object

In [14]:
data.min()

ProductFamily_ID                        1
ProductCategory_ID                      1
ProductBrand_ID                         1
ProductName_ID                          1
ProductPackSKU_ID                       1
Point-of-Sale_ID                        1
Date                  2016-01-01 00:00:00
Measures                            False
Value                                 -10
dtype: object

In [15]:
data.to_pickle('BC5DataNoString.pkl')

In [16]:
# load from here

In [17]:
data = pd.read_pickle('BC5DataNoString.pkl')

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182342304 entries, 0 to 182342303
Data columns (total 9 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ProductFamily_ID    uint8         
 1   ProductCategory_ID  uint8         
 2   ProductBrand_ID     uint16        
 3   ProductName_ID      uint16        
 4   ProductPackSKU_ID   uint16        
 5   Point-of-Sale_ID    uint16        
 6   Date                datetime64[ns]
 7   Measures            bool          
 8   Value               float64       
dtypes: bool(1), datetime64[ns](1), float64(1), uint16(4), uint8(2)
memory usage: 4.6 GB


In [19]:
data.Measures.value_counts()

True     91171152
False    91171152
Name: Measures, dtype: int64

In [20]:
data.head()

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  ProductName_ID  \
0                16                  11              306             649   
1                16                  11              306             649   
2                16                  11              306             649   
3                16                  11              306             649   
4                16                  11              306             649   

   ProductPackSKU_ID  Point-of-Sale_ID       Date  Measures   Value  
0               1970                 1 2017-03-04     False     2.0  
1               1970                 1 2017-03-04      True  1540.0  
2               1970                 1 2016-05-02     False     4.0  
3               1970                 1 2016-05-02      True  3080.0  
4               1970                 1 2016-10-24     False     2.0

In [21]:
data.memory_usage()

Index                        128
ProductFamily_ID       182342304
ProductCategory_ID     182342304
ProductBrand_ID        364684608
ProductName_ID         364684608
ProductPackSKU_ID      364684608
Point-of-Sale_ID       364684608
Date                  1458738432
Measures               182342304
Value                 1458738432
dtype: int64

# 3. Product Name - Category - Family Association

This section hopes to check if a product(name) can belong to multiple family/category, and whether a category can belong to multiple family, etc. Say, a category belongs to only one family, we can create separate the family information from the main data, apply necessary computation/tranformation on the main data and later retrieve the family information. This analysis was done on a separate notebook to avoid memory overload. Please refer to "BC5_2 Product Group Level Relationships.ipynb"

**Conclution:** a category can belong to multiple families, a product(name) can belong to multiple categories and families.

# 4. Data Aggregation

## 4.1 Data Aggregation Prep 1 - Duplicate Analysis

In [22]:
data_nodup = data.drop_duplicates()
data_nodup.Measures.value_counts()

True     91168491
False    91167021
Name: Measures, dtype: int64

In [23]:
# Save memory
del data_nodup

**Consideration**

- Unit and Price (Measures = False / True, respectively) are in pairs, hence the number should match, as in the original data, both being 91171152. However, the numbers would differ after duplicate removal.
- This may indicate some bigger underlying data integrity issue.

It's known that:
1. Measure "Sell-out units" and "Sell-out values" come in pairs
 - During the presentation, Vasco clarified that whether the price changes mid-day or not, all the units sold and all the dollar values are aggregated per PoS per SKU per day.
 - (To rephrase, there should be one dollar value and one units value for each SKU for each PoS in a given day)
2. The data provide agrees the point above.
 - There are equal amount of rows for sell-out units and sell-out values: 91171152, to be precise.

But upon removal of duplicates, there's a disproportionate reduction of sell-out values and sell-out units to 91168491 and 91167021, respectively.

There were two types of duplicates identified.
Type 1 is repetition of both value and units as boxed below, using PoS 374 as a sample.

In [24]:
pd.set_option('display.max_rows', 100)
duple = data[data.duplicated(keep='first')].sort_values(by='Date')
duple

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  \
165591788                15                   3              828   
40986350                 15                   3              828   
155846632                15                   3              828   
165591789                15                   3              828   
125707763                 1                 178              985   
...                     ...                 ...              ...   
145885198                21                 109              156   
15022910                 16                  11             1387   
98682258                  9                 178             1273   
98682259                  9                 178             1273   
99015472                  1                 178              929   

           ProductName_ID  ProductPackSKU_ID  Point-of-Sale_ID       Date  \
165591788            1581               4835               374 2016-01-02   
40986350             1581               4835                80 2016-01-02   
155846632            1581               4835               354 2016-01-02   
165591789            1581               4835               374 2016-01-02   
125707763            1846               5594               282 2016-01-03   
...                   ...                ...               ...        ...   
145885198             324               1121               331 2019-10-30   
15022910             2569               7890                30 2019-10-31   
98682258             2379               7331               218 2019-11-01   
98682259             2379               7331               218 2019-11-01   
99015472             1762               5329               219 2019-11-01   

           Measures  Value  
165591788     False    1.0  
40986350      False    1.0  
155846632     False    1.0  
165591789      True  660.0  
125707763      True  397.0  
...             ...    ...  
145885198     False    1.0  
15022910      False    1.0  
98682258      False    1.0  
98682259       True  479.0  
99015472      False    1.0  

[6792 rows x 9 columns]

In [25]:
duple.to_pickle('BC5Duple.pkl')

In [26]:
test = data.loc[data['Point-of-Sale_ID'] == 374] # Check the first row of "duple" dataframe above.
test.Measures.value_counts()

True     256365
False    256365
Name: Measures, dtype: int64

In [27]:
dup = test.duplicated() # identify the first occurence of the duplicate (keep = 'first' by default)
dup = pd.DataFrame(dup)
dup[dup[0] == True]

0
165349083  True
165349087  True
165437358  True
165437372  True
165437572  True
165437586  True
165447536  True
165447538  True
165451402  True
165452405  True
165452420  True
165452928  True
165452947  True
165469120  True
165469123  True
165591786  True
165591788  True
165591789  True
165591792  True
165591796  True

In [28]:
test2 = test[test.duplicated(keep=False)].sort_values(by='Date') # keep = False to show both first instance and its duplicated entry
test2.Measures.value_counts()

False    28
True     12
Name: Measures, dtype: int64

In [29]:
test2

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  \
165372353                15                   3              828   
165591789                15                   3              828   
165591788                15                   3              828   
165372348                15                   3              828   
165452420                 9                 178              127   
165363070                 9                 178              127   
165451402                 9                 178              127   
165363166                 9                 178              127   
165452947                 9                 178              127   
165363107                 9                 178              127   
165372666                15                   3              828   
165591796                15                   3              828   
165591786                15                   3              828   
165372279                15                   3              828   
165591792                15                   3              828   
165372535                15                   3              828   
165363066                 9                 178              127   
165452405                 9                 178              127   
165155758                 1                 178              933   
165437372                 1                 178              933   
165437358                 1                 178              933   
165155754                 1                 178              933   
165363104                 9                 178              127   
165452928                 9                 178              127   
165437586                 1                 178              933   
165437572                 1                 178              933   
165155791                 1                 178              933   
165155794                 1                 178              933   
165300366                 1                 178              991   
165349087                 1                 178              991   
165349083                 1                 178              991   
165300367                 1                 178              991   
165469123                12                 178              788   
165225984                12                 178              788   
165225985                12                 178              788   
165469120                12                 178              788   
165284736                12                 178              243   
165447538                12                 178              243   
165447536                12                 178              243   
165284737                12                 178              243   

           ProductName_ID  ProductPackSKU_ID  Point-of-Sale_ID       Date  \
165372353            1581               4835               374 2016-01-02   
165591789            1581               4835               374 2016-01-02   
165591788            1581               4835               374 2016-01-02   
165372348            1581               4835               374 2016-01-02   
165452420             231                754               374 2016-02-08   
165363070             231                754               374 2016-02-08   
165451402             231                755               374 2016-02-20   
165363166             231                755               374 2016-02-20   
165452947             231                754               374 2016-02-25   
165363107             231                754               374 2016-02-25   
165372666            1581               4835               374 2016-03-10   
165591796            1581               4835               374 2016-03-10   
165591786            1581               4835               374 2016-03-28   
165372279            1581               4835               374 2016-03-28   
165591792            1581               4835               374 2016

**Consideration**
Two types of "duplicates" are identified from above.
 - Type 1: Both unit value and dollar value are repeated (the first 4 rows above, "test2")
 - Type 2: Only unit value is repeated. (several subsequent rows that follow afterwards. Highlighted below.)

In [30]:
test.loc[((test['Date'] =='2016-02-08') & (test['ProductPackSKU_ID'] ==754))]

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  \
165363070                 9                 178              127   
165363077                 9                 178              127   
165452420                 9                 178              127   
165452425                 9                 178              127   

           ProductName_ID  ProductPackSKU_ID  Point-of-Sale_ID       Date  \
165363070             231                754               374 2016-02-08   
165363077             231                754               374 2016-02-08   
165452420             231                754               374 2016-02-08   
165452425             231                754               374 2016-02-08   

           Measures  Value  
165363070     False    1.0  
165363077      True  490.0  
165452420     False    1.0  
165452425      True  430.0

In [31]:
test.loc[((test['Date'] =='2016-02-20') & (test['ProductPackSKU_ID'] ==755))]

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  \
165363166                 9                 178              127   
165363167                 9                 178              127   
165451402                 9                 178              127   
165451411                 9                 178              127   

           ProductName_ID  ProductPackSKU_ID  Point-of-Sale_ID       Date  \
165363166             231                755               374 2016-02-20   
165363167             231                755               374 2016-02-20   
165451402             231                755               374 2016-02-20   
165451411             231                755               374 2016-02-20   

           Measures  Value  
165363166     False    1.0  
165363167      True  440.0  
165451402     False    1.0  
165451411      True  435.0

These entries appear to be legitimate. Hence should be aggregated, but will be done at the end, after reducing the memory usage of the data a little more.

**Update** These "duplicate" elements were confirmed by our client to be a result of incomplete aggregation. Hence, the decision to aggregate them is correct.

## 4.2 Data Aggregation Prep 2 - Value variable

### 4.2.1 Coherence Check

In [32]:
data.Value.max()

662575.0

In [33]:
data.Value.min()

-10.0

In [34]:
data[data['Measures']==False]['Value'].min()

1.0

In [35]:
data[data['Measures']==True]['Value'].min()

-10.0

In [36]:
data[data['Value'] <0] 
# Single instance negative value. The data integrity of Value column appears to remain largely unaffected.

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  \
111402150                21                 117             1472   

           ProductName_ID  ProductPackSKU_ID  Point-of-Sale_ID       Date  \
111402150            1229               3668               252 2017-04-19   

           Measures  Value  
111402150      True  -10.0

In [37]:
data.loc[((data['Date'] =='2017-04-19') & (data['Point-of-Sale_ID'] ==252)& (data['ProductName_ID'] ==1229))]

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  \
111402146                21                 117             1472   
111402150                21                 117             1472   
111460370                21                 115             1472   
111460375                21                 115             1472   

           ProductName_ID  ProductPackSKU_ID  Point-of-Sale_ID       Date  \
111402146            1229               3668               252 2017-04-19   
111402150            1229               3668               252 2017-04-19   
111460370            1229               3684               252 2017-04-19   
111460375            1229               3684               252 2017-04-19   

           Measures   Value  
111402146     False     2.0  
111402150      True   -10.0  
111460370     False     1.0  
111460375      True  4110.0

In [38]:
# We cannot find any indication this is a price correction entry. If it was, we should be able to locate another entry pair made for the same SKU on the same date at the same PoS.
# A common accounting practice for price correction entry would be done with quanity = 0 paired with a price correction entry. Hence, we are reliant to believe index number 111402146 and 111402150 are erroneous entries.

# Drop the pair for negative value price
data.drop(data.loc[[111402146,111402150],:].index, inplace=True)

In [39]:
# Check
data.loc[((data['Date'] =='2017-04-19') & (data['Point-of-Sale_ID'] ==252)& (data['ProductName_ID'] ==1229))]

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  \
111460370                21                 115             1472   
111460375                21                 115             1472   

           ProductName_ID  ProductPackSKU_ID  Point-of-Sale_ID       Date  \
111460370            1229               3684               252 2017-04-19   
111460375            1229               3684               252 2017-04-19   

           Measures   Value  
111460370     False     1.0  
111460375      True  4110.0

In [40]:
# Check
data[data['Measures']==True]['Value'].min()

0.0

In [41]:
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182342302 entries, 0 to 182342303
Data columns (total 9 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ProductFamily_ID    uint8         
 1   ProductCategory_ID  uint8         
 2   ProductBrand_ID     uint16        
 3   ProductName_ID      uint16        
 4   ProductPackSKU_ID   uint16        
 5   Point-of-Sale_ID    uint16        
 6   Date                datetime64[ns]
 7   Measures            bool          
 8   Value               float64       
dtypes: bool(1), datetime64[ns](1), float64(1), uint16(4), uint8(2)
memory usage: 5.9 GB


In [42]:
data[data['Value'] ==0]

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  \
2883769                  21                 159              545   
11145385                 18                 175             1142   
11145387                 18                 175             1142   
11145389                 18                 175             1142   
11145391                 18                 175             1142   
...                     ...                 ...              ...   
181736091                19                 174              940   
181736093                19                 174              940   
181736095                19                 174              940   
181736097                19                 174              940   
182225075                21                 109              363   

           ProductName_ID  ProductPackSKU_ID  Point-of-Sale_ID       Date  \
2883769              1089               3276                 6 2016-03-14   
11145385             2102               6353                23 2017-11-21   
11145387             2102               6353                23 2018-06-25   
11145389             2102               6353                23 2016-03-21   
11145391             2102               6353                23 2016-11-01   
...                   ...                ...               ...        ...   
181736091            1784               5384               409 2019-05-17   
181736093            1784               5384               409 2017-02-27   
181736095            1784               5384               409 2018-05-02   
181736097            1784               5384               409 2018-02-07   
182225075             744               2260               410 2016-04-01   

           Measures  Value  
2883769        True    0.0  
11145385       True    0.0  
11145387       True    0.0  
11145389       True    0.0  
11145391       True    0.0  
...             ...    ...  
181736091      True    0.0  
181736093      True    0.0  
181736095      True    0.0  
181736097      True    0.0  
182225075      True    0.0  

[669 rows x 9 columns]

In [43]:
data.loc[((data['Measures']==True) & (data['Value'] ==0))]

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  \
2883769                  21                 159              545   
11145385                 18                 175             1142   
11145387                 18                 175             1142   
11145389                 18                 175             1142   
11145391                 18                 175             1142   
...                     ...                 ...              ...   
181736091                19                 174              940   
181736093                19                 174              940   
181736095                19                 174              940   
181736097                19                 174              940   
182225075                21                 109              363   

           ProductName_ID  ProductPackSKU_ID  Point-of-Sale_ID       Date  \
2883769              1089               3276                 6 2016-03-14   
11145385             2102               6353                23 2017-11-21   
11145387             2102               6353                23 2018-06-25   
11145389             2102               6353                23 2016-03-21   
11145391             2102               6353                23 2016-11-01   
...                   ...                ...               ...        ...   
181736091            1784               5384               409 2019-05-17   
181736093            1784               5384               409 2017-02-27   
181736095            1784               5384               409 2018-05-02   
181736097            1784               5384               409 2018-02-07   
182225075             744               2260               410 2016-04-01   

           Measures  Value  
2883769        True    0.0  
11145385       True    0.0  
11145387       True    0.0  
11145389       True    0.0  
11145391       True    0.0  
...             ...    ...  
181736091      True    0.0  
181736093      True    0.0  
181736095      True    0.0  
181736097      True    0.0  
182225075      True    0.0  

[669 rows x 9 columns]

**Consideration:**
- All 0 Value is from sales price. 
- Unlike the negative value, this phenomenon is more likely to happen. Could be a result of a promotion, bundled set, or quantity correction entry (by inserting 0 price paired with a quantity, a common accounting practice.), etc.

**Conclusion:** We keep these entries, which will be aggregated by ProductName per Date per PoS.

### 4.2.2 Value Variable Datatype
**Consideration:** Is it worth it to convert Value to integer to save memory?

In [44]:
# Family, category and brand can be retrieved later
#data = data[['ProductName_ID','Point-of-Sale_ID','Date','Measures','Value']]

In [45]:
test = data[data['Value'] % 1 != 0]

In [46]:
test['Value'].min()

241.5

In [47]:
test['Value'].max()

290026.748251

In [48]:
test[test['Measures']==False]
# No unit sold amount in float, which is logical

Empty DataFrame
Columns: [ProductFamily_ID, ProductCategory_ID, ProductBrand_ID, ProductName_ID, ProductPackSKU_ID, Point-of-Sale_ID, Date, Measures, Value]
Index: []

In [49]:
test.memory_usage()

Index                 569048
ProductFamily_ID       71131
ProductCategory_ID     71131
ProductBrand_ID       142262
ProductName_ID        142262
ProductPackSKU_ID     142262
Point-of-Sale_ID      142262
Date                  569048
Measures               71131
Value                 569048
dtype: int64

In [50]:
test2 = test.copy()
test2['Value'] = test2['Value'].astype(np.uint32)
test2.memory_usage()

Index                 569048
ProductFamily_ID       71131
ProductCategory_ID     71131
ProductBrand_ID       142262
ProductName_ID        142262
ProductPackSKU_ID     142262
Point-of-Sale_ID      142262
Date                  569048
Measures               71131
Value                 284524
dtype: int64

In [51]:
(len(test))/(len(data)) #amount of float values = 0.04%

0.0003900959855162956

**Consideration**
- Only 0.04% of the total data is in float
- Smallest float value is 241.5. 0.5 added to round the value is only 0.2% increment, meaning the impact would be very small.
- Memory usage of Value halves when converted to uint32

**Conclution:** We convert the float values to integer in Value variable

**Value Datatype Conversion**

In [54]:
data.Value = data.Value.round()
data.Value = data.Value.astype('int32')
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182342302 entries, 0 to 182342303
Data columns (total 9 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ProductFamily_ID    uint8         
 1   ProductCategory_ID  uint8         
 2   ProductBrand_ID     uint16        
 3   ProductName_ID      uint16        
 4   ProductPackSKU_ID   uint16        
 5   Point-of-Sale_ID    uint16        
 6   Date                datetime64[ns]
 7   Measures            bool          
 8   Value               int32         
dtypes: bool(1), datetime64[ns](1), int32(1), uint16(4), uint8(2)
memory usage: 5.3 GB


In [55]:
# Save memory
del dup
del test
del test2
del duple #saved as a file previously
del objects

# 4.3 Data Aggregation

In [56]:
# The client only wishes for ProductName level Analysis at most, and we have already dealt with "duplicate" items, the variable is no longer needed.
del data['ProductPackSKU_ID'] 

In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182342302 entries, 0 to 182342303
Data columns (total 8 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ProductFamily_ID    uint8         
 1   ProductCategory_ID  uint8         
 2   ProductBrand_ID     uint16        
 3   ProductName_ID      uint16        
 4   Point-of-Sale_ID    uint16        
 5   Date                datetime64[ns]
 6   Measures            bool          
 7   Value               int32         
dtypes: bool(1), datetime64[ns](1), int32(1), uint16(3), uint8(2)
memory usage: 4.9 GB


In [58]:
with open('BC5Data_PreAgg.pkl', 'wb') as f:
    pickle.dump(data, f)

### Aggregation method choices
- due to memory issue, pivot_talbe and agg method is not possible, hence the operation needs to be separated
- for numpy array, np.sum is much faster, but slow on a list. 
- Pandas series is very similar to numpy array so it's worth comparing

In [59]:
%%time
data_sum = data.head(1000).groupby(['ProductName_ID','Point-of-Sale_ID','Date','Measures']).sum()
data_sum

CPU times: user 6.05 ms, sys: 1.54 ms, total: 7.59 ms
Wall time: 161 ms


ProductFamily_ID  \
ProductName_ID Point-of-Sale_ID Date       Measures                     
649            1                2016-03-29 False                   16   
                                           True                    16   
                                2016-04-19 False                   16   
                                           True                    16   
                                2016-05-02 False                   16   
...                                                               ...   
2008           1                2019-10-29 True                     1   
                                2019-10-30 False                    1   
                                           True                     1   
                                2019-11-01 False                    1   
                                           True                     1   

                                                     ProductCategory_ID  \
ProductName_ID Point-of-Sale_ID Date       Measures                       
649            1                2016-03-29 False                     11   
                                           True                      11   
                                2016-04-19 False                     11   
                                           True                      11   
                                2016-05-02 False                     11   
...                                                                 ...   
2008           1                2019-10-29 True                      33   
                                2019-10-30 False                     33   
                                           True                      33   
                                2019-11-01 False                     33   
                                           True                      33   

                                                     ProductBrand_ID  Value  
ProductName_ID Point-of-Sale_ID Date       Measures                          
649            1                2016-03-29 False                 306      2  
                                           True                  306   1540  
                                2016-04-19 False                 306      2  
                                           True                  306   1540  
                                2016-05-02 False                 306      4  
...                                                              ...    ...  
2008           1                2019-10-29 True                 1092   1180  
                                2019-10-30 False                1092      2  
                                           True                 1092   1180  
                                2019-11-01 False                1092      5  
                                           True                 1092   2790  

[1000 rows x 4 columns]

In [60]:
%%time
data_np = data.head(1000).groupby(['ProductName_ID','Point-of-Sale_ID','Date','Measures']).agg({"Value": np.sum})
data_np

CPU times: user 4.69 ms, sys: 0 ns, total: 4.69 ms
Wall time: 5.93 ms


Value
ProductName_ID Point-of-Sale_ID Date       Measures       
649            1                2016-03-29 False         2
                                           True       1540
                                2016-04-19 False         2
                                           True       1540
                                2016-05-02 False         4
...                                                    ...
2008           1                2019-10-29 True       1180
                                2019-10-30 False         2
                                           True       1180
                                2019-11-01 False         5
                                           True       2790

[1000 rows x 1 columns]

In [61]:
# memory save
del data_np
del data_sum

**Conclusion:** python basic sum() wins

**Decision:** The revenue element is already price * quantity, hence there is no reason to have both quantity and revenue in one dataframe for the purpose of our analysis.
Hence we separate the dataframe into two.

In [62]:
data = pd.read_pickle('BC5Data_PreAgg.pkl')

In [63]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182342302 entries, 0 to 182342303
Data columns (total 8 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ProductFamily_ID    uint8         
 1   ProductCategory_ID  uint8         
 2   ProductBrand_ID     uint16        
 3   ProductName_ID      uint16        
 4   Point-of-Sale_ID    uint16        
 5   Date                datetime64[ns]
 6   Measures            bool          
 7   Value               int32         
dtypes: bool(1), datetime64[ns](1), int32(1), uint16(3), uint8(2)
memory usage: 4.9 GB


In [64]:
#data = data[['ProductFamily_ID','ProductCategory_ID','ProductName_ID','Point-of-Sale_ID','Date','Measures','Value']] # Product Brand can be retrieved from product Name, as each productname belongs to exactly 1 brand.

In [65]:
data_q = data[data['Measures']==False]

In [66]:
data_q = data_q.drop('Measures', axis=1)
data_q.head()

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  ProductName_ID  \
0                16                  11              306             649   
2                16                  11              306             649   
4                16                  11              306             649   
6                16                  11              306             649   
7                16                  11              306             649   

   Point-of-Sale_ID       Date  Value  
0                 1 2017-03-04      2  
2                 1 2016-05-02      4  
4                 1 2016-10-24      2  
6                 1 2017-10-13      2  
7                 1 2017-10-14      2

In [67]:
%%time
data_q = data_q.groupby(['ProductFamily_ID','ProductCategory_ID','ProductBrand_ID','ProductName_ID','Date','Point-of-Sale_ID']).sum()
# Note, data contains "duplicates" which were found to be legitimate entries, not double entries. Hence they are aggregated as well.

CPU times: user 45.3 s, sys: 9.41 s, total: 54.7 s
Wall time: 54.7 s


In [68]:
data_q.reset_index(inplace=True)
data_q.head()

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  ProductName_ID  \
0                 1                  33               35              77   
1                 1                  33              313             657   
2                 1                  33              313             657   
3                 1                  33              313             657   
4                 1                  33              313             657   

        Date  Point-of-Sale_ID  Value  
0 2016-12-27               150      1  
1 2016-01-01                13      1  
2 2016-01-01                35      1  
3 2016-01-01                77      1  
4 2016-01-01               104      1

In [69]:
data_q.rename(columns={'Value': 'Quantity'}, inplace=True)

In [70]:
data_q.head()

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  ProductName_ID  \
0                 1                  33               35              77   
1                 1                  33              313             657   
2                 1                  33              313             657   
3                 1                  33              313             657   
4                 1                  33              313             657   

        Date  Point-of-Sale_ID  Quantity  
0 2016-12-27               150         1  
1 2016-01-01                13         1  
2 2016-01-01                35         1  
3 2016-01-01                77         1  
4 2016-01-01               104         1

In [71]:
data_r = data[data['Measures']==True]
data_r = data_r.drop('Measures', axis=1)
data_r.head()

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  ProductName_ID  \
1                 16                  11              306             649   
3                 16                  11              306             649   
5                 16                  11              306             649   
9                 16                  11              306             649   
10                16                  11              306             649   

    Point-of-Sale_ID       Date  Value  
1                  1 2017-03-04   1540  
3                  1 2016-05-02   3080  
5                  1 2016-10-24   1540  
9                  1 2017-10-13   1620  
10                 1 2017-10-14   1620

In [72]:
#%%time
data_r = data_r.groupby(['ProductFamily_ID','ProductCategory_ID','ProductBrand_ID','ProductName_ID','Date','Point-of-Sale_ID']).sum()
# Note, data contains "duplicates" which were found to be legitimate entries, not double entries. Hence they are aggregated as well.

In [73]:
data_r.reset_index(inplace=True)
data_r.head()

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  ProductName_ID  \
0                 1                  33               35              77   
1                 1                  33              313             657   
2                 1                  33              313             657   
3                 1                  33              313             657   
4                 1                  33              313             657   

        Date  Point-of-Sale_ID  Value  
0 2016-12-27               150    495  
1 2016-01-01                13    795  
2 2016-01-01                35    695  
3 2016-01-01                77    470  
4 2016-01-01               104    430

In [74]:
data_r.rename(columns={'Value': 'Revenue'}, inplace=True)

In [75]:
data_r.head()

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  ProductName_ID  \
0                 1                  33               35              77   
1                 1                  33              313             657   
2                 1                  33              313             657   
3                 1                  33              313             657   
4                 1                  33              313             657   

        Date  Point-of-Sale_ID  Revenue  
0 2016-12-27               150      495  
1 2016-01-01                13      795  
2 2016-01-01                35      695  
3 2016-01-01                77      470  
4 2016-01-01               104      430

In [76]:
len(data)

182342302

In [77]:
data.Measures.value_counts()

True     91171151
False    91171151
Name: Measures, dtype: int64

In [78]:
print(len(data_q))
print(len(data_r))

77529406
77529406


In [79]:
# Save memory
del data

In [80]:
with open('BC5Data_Quantity.pkl', 'wb') as f:
    pickle.dump(data_q, f)

In [81]:
with open('BC5Data_Revenue.pkl', 'wb') as f:
    pickle.dump(data_r, f)

# Utility

In [82]:
# Memory Usage check when needed
objects=[]
for name,obj in locals().items():
    objects.append([name,sys.getsizeof(obj)])
sorted(objects,key=lambda x: x[1],reverse=True)

[['data_q', 4031529264],
 ['data_r', 4031529264],
 ['chunk', 152249916],
 ['_24', 237744],
 ['_42', 23439],
 ['_43', 23439],
 ['_59', 16930],
 ['_60', 12930],
 ['_4', 5746],
 ['_oh', 2280],
 ['Out', 2280],
 ['tqdm', 2000],
 ['objects', 1448],
 ['_29', 1424],
 ['_9', 1387],
 ['_10', 1387],
 ['_13', 1349],
 ['_14', 1349],
 ['_21', 1112],
 ['_49', 1112],
 ['_50', 1112],
 ['HTML', 1056],
 ['_i7', 835],
 ['_ih', 768],
 ['In', 768],
 ['_i5', 668],
 ['_i11', 567],
 ['_i38', 555],
 ['___', 412],
 ['_68', 412],
 ['_70', 412],
 ['_73', 412],
 ['_75', 412],
 ['_i2', 372],
 ['_i72', 323],
 ['_i67', 322],
 ['_20', 287],
 ['_i3', 271],
 ['_i64', 266],
 ['_i56', 228],
 ['_i53', 224],
 ['_i82', 220],
 ['_i28', 211],
 ['_27', 204],
 ['_i27', 192],
 ['_', 188],
 ['_19', 188],
 ['_22', 188],
 ['_26', 188],
 ['_28', 188],
 ['_77', 188],
 ['_i36', 179],
 ['_i44', 178],
 ['_i60', 177],
 ['_i26', 176],
 ['f', 176],
 ['_i39', 166],
 ['_i4', 165],
 ['_i1', 164],
 ['_30', 164],
 ['_31', 164],
 ['_37', 164],
 ['